In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

import torch

import numpy as np

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq
from transformers import pipeline

In [4]:
NEWS_DATA_PATH = 'drive/My Drive/Colab Notebooks/期末/BBC News Summary/News Articles'
SUMMARIES_DATA_PATH = 'drive/My Drive/Colab Notebooks/期末/BBC News Summary/Summaries'

# 設定模型儲存路徑
MODEL_SAVE_PTH = 'drive/My Drive/Colab Notebooks/期末/5.五個分類_2個epoch_滑動平均損失和動態調整權重'

# 設定設備為 GPU (如果可用的話) 或 CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 設定輸入和目標(摘要)的最大長度
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

# 設定批次大小
BATCH_SIZE = 8

# 定義情緒標籤
SENTIMENT_LABELS = ["happy", "sad", "warn", "angry", "sorrow", "alert", "neutral"]

# 建立情緒標籤到索引的對應字典
SENTIMENT_LABELS_MAP = {label: index for index, label in enumerate(SENTIMENT_LABELS)}



# 加入權重
summary_loss_weight = 0.5
sentiment_loss_weight = 0.5
prev_summary_loss = None
prev_sentiment_loss = None
avg_summary_loss = 0
avg_sentiment_loss = 0

**從root_path 加載數據**

根路徑下的每個子目錄(代表一個類別)，子目錄中的每個文件包含新聞內容。

返回一個包含所有新聞內容的 numpy 字符串數組。

News Articles：包含不同類別的新聞文章，每個類別是一個子資料夾，內有多個文件。

Summaries：包含每篇新聞文章的五個摘要。

pbar = tqdm(os.listdir(news_dir_path))：

這行程式碼將 news_dir_path 目錄中的所有文件名稱傳遞給 tqdm，以初始化進度條 pbar。在迴圈中使用 pbar 可以在迴圈每次迭代時更新進度條。



*   os.listdir(root_path) 返回根目錄下的所有子目錄名稱，假設每個子目錄代表一個類別。
*   os.path.join(root_path, cls) 獲取每個子目錄的完整路徑。


In [5]:
# 定義資料載入函數
def load_data(root_path):
    print('Loading: {}'.format(root_path))  # 打印正在載入的路徑

    data = []  # 初始化數據列表，用來存儲加載的數據
    classes = sorted(os.listdir(root_path))  # 確保類別按順序排序
    # 遍歷根目錄下的每個子目錄，假設每個子目錄代表一個類別。
    # for cls in os.listdir(root_path):
    for cls in classes:  # 確保類別的順序一致
        news_dir_path = os.path.join(root_path, cls)  # 獲取類別資料夾的路徑

        files = sorted(os.listdir(news_dir_path))  # 確保文件按順序排序

        pbar = tqdm(files)   # 初始化進度條，用來顯示加載每個子目錄中文件的進度。

        # 遍歷類別資料夾中的文件
        for news_file_name in pbar:
            news_file_path = os.path.join(news_dir_path, news_file_name)  # 獲取新聞文件的路徑

            # 打開新聞文件並讀取內容
            with open(news_file_path, 'r', encoding='unicode_escape') as file:
                lines = file.read().strip()
                if lines:
                    data.append(lines.replace('\n', ' '))  # 將內容中的換行符替換為空格，並添加到數據列表中。
                else:
                    print(f"Empty file found: {news_file_path}")

            # 更新進度條的後綴資訊
            if news_file_name == pbar.iterable[-1]: # 如果當前文件是目錄中的最後一個文件，更新進度條的後綴信息。
                pbar.set_postfix_str('{} class loaded'.format(cls)) # 設置進度條的後綴信息，顯示當前類別已加載完成。

    return np.array(data, dtype=str)  # 返回數據陣列

In [6]:
# 載入新聞語料
news_corpus = load_data(NEWS_DATA_PATH)

# 載入摘要語料
summaries_corpus = load_data(SUMMARIES_DATA_PATH)

Loading: drive/My Drive/Colab Notebooks/期末/BBC News Summary/News Articles


100%|██████████| 401/401 [00:34<00:00, 11.61it/s, tech class loaded]


Loading: drive/My Drive/Colab Notebooks/期末/BBC News Summary/Summaries


100%|██████████| 401/401 [00:18<00:00, 21.33it/s, tech class loaded]


In [7]:
# 分割資料集為訓練集和測試集
# 5% 的資料用作測試集，剩下的 95% 用作訓練集
X_train, X_test, y_train, y_test = train_test_split(news_corpus, summaries_corpus, test_size=0.05, random_state=42)

# 將訓練集進一步分割為訓練集和驗證集
# 之前的訓練集再一次分割成新的訓練集和驗證集。
# test_size=0.05 表示將 5% 的訓練集數據用作驗證集，剩下的 95% 繼續用作訓練集。
X_train, X_vaild, y_train, y_vaild = train_test_split(X_train, y_train, test_size=0.05, random_state=42)

In [8]:
# 計算測試集、驗證集和訓練集的長度
len(X_test), len(X_vaild), len(X_train)

(112, 106, 2007)

**自定義 PyTorch 的 Dataset 類，用於處理新聞和摘要數據。**

初始化數據集，包括輸入數據（新聞）和目標數據（摘要）。
定義數據集的大小和索引操作。

返回值：

    __len__ 返回數據集的大小。

    __getitem__ 根據索引返回對應的數據對（新聞和摘要）。

In [9]:
# 定義資料集類別
class CorpusDataset(Dataset):
    def __init__(self, X_corpus, y_corpus):
        self.X_corpus = X_corpus  # 輸入數據
        self.y_corpus = y_corpus  # 目標數據

    def __len__(self):
        return len(self.X_corpus)  # 返回數據集的大小

    def __getitem__(self, index):
        return self.X_corpus[index], self.y_corpus[index]  # 根據索引返回對應的數據對


In [10]:
# 創建訓練、驗證和測試資料集
train_model_inputs = CorpusDataset(X_train, y_train)
vaild_model_inputs = CorpusDataset(X_vaild, y_vaild)
test_model_inputs = CorpusDataset(X_test, y_test)

1. 程式碼初始化已定義 BATCH_SIZE = 8


2. drop_last=True

*   如果最後一個批次數據不夠，則丟棄
*   如果最後一個批次數據不夠，則丟棄















pinned memory（固定內存）

 page-locked memory。內存的數據不會被交換到磁盤上（即不會被置換出內存），因此 GPU 可以更快地訪問和複製這些數據。

*   GPU 進行深度學習訓練，需要將數據從 CPU 內存複製到 GPU 內存
*   CPU 固定內存（pinned memory）確保資料不會被置換出內存
*   更高效的 DMA（Direct Memory Access，直接內存訪問）操作
*   載入資料DataLoader 會將數據預先加載到 CPU 的固定內存區域，可以更快地傳輸到 GPU 內存中進行計算

**什麼情況下使用** `pin_memory=True` 有幫助

1. **頻繁數據傳輸**：需要頻繁地將大批量數據從 CPU 傳輸到 GPU，例如在每個訓練批次中，使用 `pin_memory=True` 可以顯著提高數據傳輸速度。
   
2. **大數據量**：當處理的數據量非常大時，固定內存可以減少數據傳輸過程中的瓶頸。

3. **使用GPU**：當使用 GPU 進行訓練時，`pin_memory=True` 尤其有幫助，可以加速數據從 CPU 到 GPU 的傳輸。

使用 `pin_memory=True` 時，確實可以提高數據從 CPU 傳輸到 GPU 的速度，但這並不意味著在所有情況下都能顯著提升性能。

1. **內存消耗**：使用 `pin_memory=True` 會佔用更多的 CPU 內存，如果你的系統內存有限，這可能會導致內存不足的情況。

2. **小批次數據**：如果你的批次大小很小，或者數據傳輸的頻率不高，使用 `pin_memory=True` 可能不會帶來顯著的性能提升。

3. **數據加載瓶頸**：`pin_memory=True` 主要優化的是數據從 CPU 到 GPU 的傳輸速度。如果你的數據加載過程（例如從磁盤讀取數據）是主要瓶頸，這個選項不會對這部分的性能產生影響。

**內存資源允許的情況下，這是一個很好的選擇，可以減少數據傳輸的延遲，提升整體訓練效率。**

**然而，對於小批次數據或內存有限的環境，提升效果可能不明顯。**


In [11]:
# 創建訓練資料加載器
train_dataloader = DataLoader(
    train_model_inputs,
    # 避免局部最小
    shuffle=True,  # 訓練時打亂數據
    batch_size=BATCH_SIZE,  # 每個批次的大小
    drop_last=True,  # 如果最後一個批次數據不夠，則丟棄
    pin_memory=True  # 將數據複製到 CUDA 固定內存
)

# 創建驗證資料加載器
eval_dataloader = DataLoader(
    vaild_model_inputs,
    shuffle=False,  # 驗證時不打亂數據
    batch_size=BATCH_SIZE,  # 每個批次的大小
    drop_last=True,  # 如果最後一個批次數據不夠，則丟棄
    pin_memory=True  # 將數據複製到 CUDA 固定內存
)

# 創建測試資料加載器
test_dataloader = DataLoader(
    test_model_inputs,
    shuffle=False,  # 測試時不打亂數據
    batch_size=1,  # 每個批次的大小為1
    drop_last=True,  # 如果最後一個批次數據不夠，則丟棄
    pin_memory=True  # 將數據複製到 CUDA 固定內存
)

初始化:設定設備為 GPU (如果可用的話) 或 CPU

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


*   torch.cuda.is_available() 檢查當前系統是否有可用的 GPU。
*   有可用的 GPU（torch.cuda.is_available() 返回 True），則 DEVICE 被設置為 torch.device("cuda")，這表示後續的運算將在 GPU 上進行。
*   沒有可用的 GPU，則 DEVICE 被設置為 torch.device("cpu")，這表示後續的運算將在 CPU 上進行。




**pipeline 支持多種 NLP 任務，自動處理文本標記化和模型推理。**

使用方法：pipeline("任務名稱")

**零樣本分類（zero-shot classification） 可以在沒有專門針對目標類別進行訓練的情況下進行分類。**

*   透過自然語言推理（Natural Language Inference, NLI）判斷輸入文本與候選標籤之間的關係，語義理解來進行分類


**已經定義好的情緒分析模型（如 TextBlob 或 VADER），不需要使用零樣本分類**


In [12]:
# 加載 BART 摘要生成模型的標記器
# 'facebook/bart-large-cnn' 模型
summary_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn', device=DEVICE)

# 加載並初始化 BART 摘要生成模型
# 生成條件摘要
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(DEVICE)

# 加載並初始化 BART 的情感分類模型
sentiment_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=DEVICE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

**train_dataloader用於將訓練數據分成多個小批次**

訓練數據集有 1000 個樣本，每個批次大小為 32，則 train_dataloader 的長度約為 1000 / 32 = 31.25，這裡取整後即為 31 或 32。

**num_update_steps_per_epoch：每個週期中需要更新模型參數的步數**

**num_training_steps：表示總的訓練步數，即所有訓練週期中的總更新次數。**







**loss_fcn = CrossEntropyLoss()**
輸入的預測分布（logits 或 softmax probabilities）：

形狀：(𝑁,𝐶)

*   𝑁是批次大小。

*   𝐶是每個輸入樣本的類別數。


每個元素是模型對某一類別的預測值（logits）。這些值可以是未經 softmax 函數轉換的原始 logits，也可以是已經轉換過的概率值。

目標標籤（ground truth labels）：

形狀：(𝑁)每個元素是對應於輸入樣本的真實類別索引（長度為 𝑁，每個值是
0 到 𝐶−1 之間的整數）。

**num_train_epochs**

In [17]:
# 設定訓練的週期數，每個週期包括對整個訓練數據集的一次完整遍歷。
num_train_epochs = 2

# 計算每個週期的更新步數
num_update_steps_per_epoch = len(train_dataloader)
# 總的訓練步數
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# 定義數據整理器
# **DataCollatorForSeq2Seq 是專門為序列到序列（Seq2Seq），包含標記化、填充
data_collator = DataCollatorForSeq2Seq(summary_tokenizer, model=summary_model)

# 定義優化器
# **summary_model.parameters()：將 summary_model 的所有參數傳遞給優化器
optimizer = AdamW(summary_model.parameters(), lr=2e-5)

# 損失函數
# 計算模型預測結果與實際標籤之間的差異
loss_fcn = CrossEntropyLoss()

**get_model_inputs 函數:將摘要文本轉換為模型的目標輸入格式**




*   tokenizer：用於標記化文本的 tokenizer，通常是預訓練模型的 tokenizer，例如 BART 或 BERT。
*   corpus：一個包含多個文本的列表，每個文本都需要進行標記化處理。
*   is_summaries：一個布爾值，指示是否處理的是摘要。如果是摘要，則使用不同的最大長度。


In [18]:
# 定義資料處理函數
def get_model_inputs(tokenizer, corpus, is_summaries=False):
    model_inputs = []  # 初始化模型輸入列表

    # 設定最大長度，摘要用 MAX_TARGET_LENGTH，其他用 MAX_INPUT_LENGTH
    # MAX_TARGET_LENGTH與MAX_INPUT_LENGTH在最前面定義了
    max_length = MAX_INPUT_LENGTH if not is_summaries else MAX_TARGET_LENGTH

    # 對每個文本進行標記
    for text in corpus:
        model_inputs.append(tokenizer(
            text,
            max_length=max_length,
            truncation=True,  # 超過最大長度時進行截斷
        ))

    return model_inputs  # 返回標記化的輸入數據

# labels 是經過 get_model_inputs 函數和 data_collator 函數處理後的輸出
# 包含了轉換為標記 ID 的摘要文本（input_ids）和相應的注意力掩碼（attention_mask）。
# 這些張量將被用作目標數據，傳遞給模型用於計算損失。

'''
像是
{
    'input_ids': tensor([[101, 2023, 2003, 1037, 7099,  ...], [101, 2129, 2079, 2017, 2424, ...]]),
    'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, ...], [1, 1, 1, 1, 1, 1, 1, ...]])
}

'''

"\n像是\n{\n    'input_ids': tensor([[101, 2023, 2003, 1037, 7099,  ...], [101, 2129, 2079, 2017, 2424, ...]]),\n    'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, ...], [1, 1, 1, 1, 1, 1, 1, ...]])\n}\n\n"

**get_sentiment_probs(sentiment_model, texts)**

情感分析模型 sentiment_model 對文本 texts 進行情感分類，獲取"**情感概率**"分數。

*   遍歷每個文本，使用情感分析模型對其進行分類。
*   獲取每個情感標籤的概率分數。

返回一個包含每個文本情感概率分數的張量。

**get_sentiment_labels(sentiment_model, texts)**

情感分析模型 sentiment_model 對文本 texts 進行情感分類，獲取"**情感標籤**"

*   遍歷每個文本，使用情感分析模型對其進行分類。
*   將分類結果映射為情感標籤。

返回一個包含每個文本情感標籤的張量。


**with torch.no_grad() 反向傳播（Backpropagation）是計算梯度並更新模型參數的過程。當我們只需要進行推理（inference）或評估模型而不需要進行訓練時，我們可以禁用梯度計算以節省內存和計算資源。**

torch 提供的功能和方法，包括定義張量、進行數學運算、自動微分等。

In [52]:
def get_sentiment_logits(sentiment_model, corpus):
    corpus_logits = []  # 存儲每個文本的情感 logits。

    with torch.no_grad():  # 不計算梯度(不需要進行反向傳播)，節省內存和計算資源
        for text in corpus:
            if text:  # 新增檢查空文本的邏輯
              sentiment = sentiment_model(text, return_all_scores=True)  # 獲取情感分析結果，返回所有分數。
              logits = [score['score'] for score in sentiment]  # 獲取每個情感標籤的分數作為 logits。
              corpus_logits.append(torch.tensor(logits))  # 將 logits 轉換為張量並添加到列表
            else:
                print(f"Empty text found in corpus")  # 新增警告信息
    return torch.stack(corpus_logits)  # 將所有 logits 張量堆疊成一個張量


'''
CrossEntropyLoss 的預期輸入（即 logits）
但 gen_summaries_sentiment_probs 經過 softmax 轉換為概率分數。
不符合CrossEntropyLoss 的預期輸入（即 logits）
'''
# 定義情感機率，返回每個情感標籤的概率分數。
def get_sentiment_probs(sentiment_model, corpus):
    corpus_scores = [] # 存儲每個文本的情感分數。

    with torch.no_grad():  # 不計算梯度(不需要進行反向傳播)，節省內存和計算資源
        for text in corpus:
            sentiment = sentiment_model(text, SENTIMENT_LABELS)  # 獲取情感分析結果，返回一個包含情感標籤和分數的字典。
            labels = sentiment['labels']
            scores = sentiment['scores']
            sorted_scores = [scores[labels.index(label)] for label in SENTIMENT_LABELS]  # 按情感標籤順序排列分數
            corpus_scores.append(torch.tensor(sorted_scores))  # 將分數轉換為張量並添加到列表

    return torch.stack(corpus_scores)  # 將所有分數張量堆疊成一個張量

# 將機率轉為情感標籤
def get_sentiment_labels(sentiment_model, corpus):
    corpus_scores = []

    with torch.no_grad():  # 禁用梯度計算，節省內存和計算資源
        for text in corpus:
            if text:  # 新增檢查空文本的邏輯
              sentiment = sentiment_model(text, SENTIMENT_LABELS)  # 使用情感模型進行情感分析，輸出結果包含各情感標籤的分數。
              label = sentiment['labels'][0]  # 獲取最高機率的情感標籤
              corpus_scores.append(SENTIMENT_LABELS_MAP[label])  # 將獲取到的情感標籤轉換為數字並添加到列表
            # SENTIMENT_LABELS_MAP 是一個字典，將情感標籤轉換為數字。
            else:
                print(f"Empty text found in corpus")  # 新增警告信息
    return torch.tensor(corpus_scores)  # 將所有標籤轉換為張量

**(自己)測試**

In [19]:
import torch
import torch.nn.functional as F

def labels_to_probs(labels, num_classes):
    """
    將標籤轉換為one-hot機率
    """
    one_hot = F.one_hot(labels, num_classes).float()
    return one_hot

def probs_to_logits(probs):
    """
    將機率轉換為logits
    """
    logits = torch.log(probs + 1e-9)  # 加上1e-9以避免log(0)
    return logits

In [20]:
def get_sentiment_probs(sentiment_model, corpus):
    corpus_scores = []
    with torch.no_grad():
        for text in corpus:
            if text:
                try:
                    sentiment = sentiment_model(text, SENTIMENT_LABELS)
                    if sentiment:
                        labels = sentiment['labels']
                        scores = sentiment['scores']
                        sorted_scores = [scores[labels.index(label)] for label in SENTIMENT_LABELS]
                        corpus_scores.append(torch.tensor(sorted_scores))
                    else:
                        print(f"Error: Sentiment model returned invalid result for text: {text}")
                        corpus_scores.append(torch.ones(len(SENTIMENT_LABELS)) / len(SENTIMENT_LABELS))  # 使用均等張量作為默认值
                except Exception as e:
                    print(f"Error processing text: {text}, error: {e}")
                    corpus_scores.append(torch.ones(len(SENTIMENT_LABELS)) / len(SENTIMENT_LABELS))  # 使用均等張量作為默认值
            else:
                print(f"Error: Empty text found in corpus")
                corpus_scores.append(torch.ones(len(SENTIMENT_LABELS)) / len(SENTIMENT_LABELS))  # 使用均等張量作為默认值
    return torch.stack(corpus_scores)

def get_sentiment_labels(sentiment_model, corpus):
    corpus_scores = []
    with torch.no_grad():
        for text in corpus:
            if text:
                try:
                    sentiment = sentiment_model(text, SENTIMENT_LABELS)
                    if sentiment and 'labels' in sentiment and len(sentiment['labels']) > 0:
                        label = sentiment['labels'][0]
                        corpus_scores.append(SENTIMENT_LABELS_MAP.get(label, -1))  # 使用 -1 作为未知标签
                    else:
                        print(f"Error: Sentiment model returned invalid result for text: {text}")
                        corpus_scores.append(-1)
                except Exception as e:
                    print(f"Error processing text: {text}, error: {e}")
                    corpus_scores.append(-1)
            else:
                print(f"Error: Empty text found in corpus")
                corpus_scores.append(-1)
    return torch.tensor(corpus_scores)


**生成文本摘要，並比較摘要與原始新聞的情感標籤是否一致，然後顯示一致的比例。**

**pbar = tqdm(test_dataloader)：初始化進度條，用於顯示測試過程中的進度。**


*   set_description 或 set_description_str 方法來設置進度條的描述信息

  pbar.set_description_str('[Epoch {}] Training'.format(epoch))
*   set_postfix_str 後綴信息來顯示一些動態數據，例如當前批次的損失和平均損失

  pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(batch_loss, avg_loss))


*   自定義進度條的顯示格式，例如修改進度條的寬度、顏色和樣式

  tqdm(total=100, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')






**test_dataloader 是測試數據的加載器。**

.to(DEVICE)
          將 input_ids 張量移動到指定的設備上，以便利用 GPU 加速運算。如果有 GPU 可用就使用 GPU，否則使用 CPU。

          # 檢查是否有可用的 CUDA 設備（即 GPU），如果有，就將 DEVICE 設置為 "cuda"；如果沒有，就設置為 "cpu"。
          DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


****3. 標準化損失和動態調整權重****

In [21]:
def adjust_weights(avg_summary_loss, avg_sentiment_loss, min_step=0.1):
    if avg_summary_loss > avg_sentiment_loss:
        return max(0.5 + min_step, 0.9), min(0.5 - min_step, 0.1)
    else:
        return min(0.5 - min_step, 0.1), max(0.5 + min_step, 0.9)

def standardize_loss(loss, mean_loss):
    if mean_loss == 0:
        return loss
    standardized_loss = (loss - mean_loss) / (abs(mean_loss) + 1e-9)
    return standardized_loss


**檢查數據集中情感標籤的分布是否均衡。**

In [ ]:
from collections import Counter
from tqdm import tqdm

def check_sentiment_distribution(corpus):
    sentiment_labels = get_sentiment_labels(sentiment_model, corpus)
    sentiment_counts = Counter(sentiment_labels.tolist())
    print("Sentiment distribution:")
    for label, count in sentiment_counts.items():
        print(f"{label}: {count}")
    return sentiment_counts

# 假設 train_dataloader 已經存在
all_news = []
for news, _ in tqdm(train_dataloader, desc="Collecting all news"):
    all_news.extend(news)

check_sentiment_distribution(all_news)


KeyboardInterrupt: 

**測試與評估模型在測試數據集上的性能。**

*   遍歷測試數據集，對每個批次進行處理：
    *   生成模型輸入。
    *   生成摘要並獲取情感標籤。
    *   打印生成的摘要和情感標籤。

評估模型在測試數據集上的生成效果。

**BART生成文本時，模型生成的 ID 序列是指模型在每個時間步生成的標記（token）的整數表示。這些 ID 序列是根據模型的詞彙表（vocabulary）進行編碼的，表示生成的每個標記在詞彙表中的索引。**


*   詞彙表可能包含以下映射：{"hello": 0, "world": 1,"\<s>": 2, "\</s>": 3, ...}。
*   "\<s> Deep learning is great \</s>"模型生成的 ID 序列可能是 [2, 451, 987, 123, 3]，其中每個整數對應於詞彙表中的一個標記。



**loss = summary_model(**).loss**

使用方法及參數可以看


*   官方文檔，例如 Hugging Face Transformers 文檔。

*   查閱模型的源碼，通常可以在模型類的 forward 方法中找到所需的參數。



In [65]:
def test_step():
    # eval() 是 torch.nn.Module 類的一個方法，用於將模型設置為評估模式。
    summary_model.eval()  # 設置模型為評估模式

    result = []

    # tqdm: 顯示迴圈的進度條
    # test_dataloader: DataLoader迭代測試數據集，能夠在小批次中加載數據
    pbar = tqdm(test_dataloader)  # 初始化進度條

    for index, (news, _) in enumerate(pbar): # 遍歷測試數據加載器中的每個批次數據。news 是新聞文本，忽略其對應的標籤 _。
        # 獲取輸入數據的ID並移動到設備
        '''
        *data_collator
          是一個函數或對象，用於將多個樣本整理成一個批次（batch）。
          在 NLP 任務中，這通常包括填充（padding）不同長度的序列以使它們具有相同的長度，以及創建注意掩碼。
          data_collator 返回的結果通常是一個字典，其中包括多個鍵，例如 input_ids、attention_mask 等。
        *['input_ids']
          模型的實際輸入數據，即每個詞語對應的 token IDs
        *.to(DEVICE)
          將 input_ids 張量移動到指定的設備上，以便利用 GPU 加速運算。
        '''
        summary_model_inputs = data_collator(get_model_inputs(summary_tokenizer, news))['input_ids'].to(DEVICE)
        # 生成摘要
        # generate 是 transformers 庫中預訓練模型的內建方法
        # 輸出是一個包含 token IDs 的張量或列表。這些 token IDs 是模型生成的摘要。
        summary_model_outputs = summary_model.generate(summary_model_inputs)

        # 解碼生成的摘要文本
        '''batch_decode 是 tokenizer 的一個方法，用於將一批 token IDs 解碼為對應的文本。
            skip_special_tokens=True 跳過特殊標記（special tokens），如 <s>, </s>, <pad> 等。
        '''
        summarized_text = summary_tokenizer.batch_decode(summary_model_outputs, skip_special_tokens=True)

        # 獲取新聞的情感標籤
        news_sentiment_label = get_sentiment_labels(sentiment_model, news)[0]
        # 獲取摘要的情感標籤
        summarized_sentiment_label = get_sentiment_labels(sentiment_model, summarized_text)[0]
        # 比較新聞和摘要的情感標籤是否一致
        # 新聞和摘要的情感如果一樣就是1，不一樣是0
        result.append(1 if news_sentiment_label == summarized_sentiment_label else 0)

        # 更新進度條後綴，顯示相同情感標籤的比例
        # 當index是test_dataloader的最後一個要素。
        if index == len(test_dataloader)-1:
            '''進度條的後綴中顯示 "Same Sentiment Rate"
               Same Sentiment Rate: result 中為 1 的比例，越高，代表生成摘要與原始新聞的情緒一致性越高
                result.count(1):計算 result 列表中值為 1 的元素數量
                代表"摘要與原始新聞的情感標籤"一致的情況
            '''
            pbar.set_postfix_str('Same Sentiment Rate: {:.3f}'.format(result.count(1) / len(result)))


**在每個訓練週期（epoch）中訓練模型**

*   生成模型輸入。
*   計算損失，包括生成摘要的損失和情感匹配損失。
*   反向傳播和參數更新。
*   記錄和打印損失。

主要作用是更新模型參數。

**計算損失並更新參數**


*   loss = summary_model(input_ids/attention_mask/labels).loss：計算訓練摘要模型的損失。
*   optimizer.zero_grad()：清空優化器的梯度。
*   loss.backward()：進行反向傳播，計算每個參數的梯度。
*   optimizer.step()：更新模型參數。




**可以試早停跟學習率與loss**

\# 自己定義EarlyStopping函數

early_stopping = EarlyStopping(patience=3, verbose=True)

\# 修改 scheduler

scheduler = get_linear_schedule_with_warmup(optimizer,
                      num_warmup_steps=total_steps * 0.1,  # 暖身步驟數
                      num_training_steps=total_steps,  # 總訓練步驟
                      last_epoch=-1)

In [ ]:
def train_step(epoch):
    # 初始化列表來存儲每個批次的訓練損失
    total_training_loss = []

    # 將模型設置為訓練模式（啟用dropout等訓練專用層）
    # BartForConditionalGeneration 模型中啟用的層(Dropout、Layer Normalization、自注意力機制、Embeddings 層)
    summary_model.train()

    # pbar = tqdm(train_dataloader): tqdm 包裝 train_dataloader 來顯示訓練進度。
    # 設置進度條的描述，顯示當前的訓練週期(epoch 數)。
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    # 訓練數據加載器中的每個批次
    for news, summaries in pbar:
        # 使用 summary_tokenizer 和 data_collator 處理新聞文本，生成模型的輸入數據。
        # 獲取訓練數據
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))

        # 將輸入的 ID 和注意力掩碼移動到設備（例如 GPU）。
        # 這些張量包含了模型的輸入數據（input IDs 和 attention masks），移動到指定（CPU 或 GPU）上，以便後續的模型計算。
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)

        # labels: 使用 summary_tokenizer 和 data_collator 處理資料集摘要，生成目標數據。
        # y_input_ids: 將目標輸入 ID 移動到設備。
        # 模型的真實摘要（target input IDs），移動到指定的設備上，以便在計算損失時使用。
        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
        y_input_ids = labels['input_ids'].to(DEVICE)


        # 計算損失並更新模型參數-> 模型生成的輸出與目標輸入 ID 計算損失
        # loss 變數:將輸入 ID、注意力掩碼和目標標籤傳遞給 summary_model (進行"前向傳播"計算)，衡量"生成的摘要與真實摘要"之間的差異
        # .loss：
          #   (在 Hugging Face 的 Transformers 庫中，當傳遞 labels 給模型時，會自動計算損失並包含在輸出中)
          #   基於目標標籤（labels）和模型預測（input_ids 和 attention_mask 的前向傳播結果）之間的交叉熵損失，差異計算出來的
        '''
        input_ids=X_input_ids：模型的輸入 ID。(將新聞轉換為模型可讀的 ID 後的結果)
        attention_mask=X_attention_mask：指示哪些位置是有效的（應該關注的），哪些是填充（padding）的
        labels=y_input_ids：模型的目標標籤(摘要文本的 ID)。(將目標摘要轉換為模型可讀的 ID 後的結果)
        '''
        loss = summary_model(input_ids=X_input_ids,
                             attention_mask=X_attention_mask,
                             labels=y_input_ids).loss


        # gen_summaries: 使用模型生成摘要文本，並解碼為可讀格式。
        '''
        summary_model.generate(X_input_ids)：這行程式碼使用模型來生成摘要
        summary_tokenizer.batch_decode：這個方法將模型生成的 ID 序列轉換回人類可讀的文本。
        '''
        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)  # 生成摘要

        # 獲取新聞的情感標籤
        '''
        get_sentiment_labels(sentiment_model, news)：
          使用情感分析模型 sentiment_model 對新聞文本 news 進行分類，獲取情感標籤(根據你定義的情感標籤映射 (SENTIMENT_LABELS_MAP))。
        .to(DEVICE)：將獲取的情感標籤張量移動到指定的設備上（DEVICE），這樣可以在訓練過程中使用 GPU 加速計算。
        '''
        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)

        # 獲取摘要的情感概率
        '''
        get_sentiment_probs(sentiment_model, gen_summaries)：
          用情感分析模型 sentiment_model 對生成的摘要 gen_summaries 進行分類，獲取情感概率分數(各個情感標籤的概率。
        .to(DEVICE)：將獲取的情感概率分數張量移動到指定的設備上（DEVICE），這樣可以在訓練過程中使用 GPU 加速計算。
        '''
        '''get_sentiment_probs輸出是機率，不符CrossEntropyLoss 的預期輸入（即 logits）'''
        '''為了要符合 CrossEntropyLoss 的預期輸入（即 logits），get_sentiment_logits返回logits'''
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        # .clamp(min=1e-9) 是一個張量操作，將 gen_summaries_sentiment_probs 的值限制在至少 1×10−9的範圍內。防止概率值過小（接近於零），以避免在後續計算中出現數值不穩定或錯誤。
        gen_summaries_sentiment_probs = gen_summaries_sentiment_probs.clamp(min=1e-9)
        gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs)

        # 加上情感匹配損失
        # 找到新聞情感的對應摘要機率，帶入損失函數做計算
        # 原本loss生成摘要與新聞的損失
        # loss_fcn基於"生成摘要的情感分布"與"原始新聞文本的情感標籤"之間的差異。
        '''輸入方式是機率，不符CrossEntropyLoss 的預期輸入（即 logits）
        loss = loss + loss_fcn(gen_summaries_sentiment_probs, news_sentiment_labels)'''
        loss = loss + loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

        '''
        每個訓練批次中梯度清除、反向傳播和模型參數更新，並且記錄和顯示訓練過程中的損失值。
        '''
        # 清除上一個批次的梯度為0
        optimizer.zero_grad()

        # 反向傳播計算梯度(鏈式法則)，基於損失值 loss 計算模型所有參數的梯度
        # 這些梯度將存儲在每個參數的 grad 屬性中。
        loss.backward()

        # 更新模型參數
        '''
        程式碼預設是Adam，通常有較好的收斂性能，並且對於不同問題的參數設定不太敏感
        '''
        optimizer.step()  # 更新參數

        # 將當前批次的損失值添加到總損失列表中
        # loss.item() 會將損失張量轉換為標量值，當前批次的損失值
        total_training_loss.append(loss.item())

        # 更新進度條後綴，顯示當前批次的損失和平均損失
        pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(loss.item(), sum(total_training_loss) / len(total_training_loss)))



**有調整loss權重的訓練過程**

In [ ]:
def train_step(epoch):
    global prev_summary_loss, prev_sentiment_loss
    total_training_loss = []
    total_summary_loss = []
    total_sentiment_loss = []

    summary_model.train()
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    for news, summaries in pbar:
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)

        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
        y_input_ids = labels['input_ids'].to(DEVICE)

        summary_loss = summary_model(input_ids=X_input_ids,
                                    attention_mask=X_attention_mask,
                                    labels=y_input_ids).loss

        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)

        if any(not gs for gs in gen_summaries):
            print(f"Empty generated summary found: {gen_summaries}")

        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        # .clamp(min=1e-9) 是一個張量操作，將 gen_summaries_sentiment_probs 的值限制在至少 1×10−9的範圍內。防止概率值過小（接近於零），以避免在後續計算中出現數值不穩定或錯誤。
        gen_summaries_sentiment_probs = gen_summaries_sentiment_probs.clamp(min=1e-9)
        gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs)
        sentiment_loss = loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)


        total_loss = summary_loss_weight * summary_loss + sentiment_loss_weight * sentiment_loss


        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_training_loss.append(total_loss.item())
        total_summary_loss.append(summary_loss.item())
        total_sentiment_loss.append(sentiment_loss.item())
        pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(
            total_loss.item(), sum(total_training_loss) / len(total_training_loss)))

    # 計算摘要損失的平均值
    avg_summary_loss = sum(total_summary_loss) / len(total_summary_loss)
    # 計算情感損失的平均值
    avg_sentiment_loss = sum(total_sentiment_loss) / len(total_sentiment_loss)

**3. 標準化損失和動態調整權重**

In [ ]:
def train_step(epoch, avg_summary_loss, avg_sentiment_loss):
    global prev_summary_loss, prev_sentiment_loss
    total_training_loss = []
    total_summary_loss = []
    total_sentiment_loss = []
    summary_model.train()
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    for news, summaries in pbar:
        if news is None or summaries is None:
            print("Error: News or Summaries is None")
            continue
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)
        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
        y_input_ids = labels['input_ids'].to(DEVICE)

        summary_loss = summary_model(input_ids=X_input_ids, attention_mask=X_attention_mask, labels=y_input_ids).loss
        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)
        if gen_summaries is None:
            print("Error: Generated summaries is None")
            continue

        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        sentiment_loss = loss_fcn(gen_summaries_sentiment_probs, news_sentiment_labels)

        # 標準化損失
        standardized_summary_loss = standardize_loss(summary_loss.item(), avg_summary_loss)
        standardized_sentiment_loss = standardize_loss(sentiment_loss.item(), avg_sentiment_loss)

        total_loss = summary_loss_weight * summary_loss + sentiment_loss_weight * sentiment_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_training_loss.append(total_loss.item())
        total_summary_loss.append(summary_loss.item())
        total_sentiment_loss.append(sentiment_loss.item())
        pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(total_loss.item(), sum(total_training_loss) / len(total_training_loss)))

    avg_summary_loss = 0.9 * avg_summary_loss + 0.1 * (sum(total_summary_loss) / len(total_summary_loss))
    avg_sentiment_loss = 0.9 * avg_sentiment_loss + 0.1 * (sum(total_sentiment_loss) / len(total_sentiment_loss))
    return avg_summary_loss, avg_sentiment_loss


**4. 滑動平均能夠平滑損失值，減少波動，使得損失值變得更加穩定和可追踪。**

In [ ]:
def train_step(epoch, avg_summary_loss, avg_sentiment_loss):
    global prev_summary_loss, prev_sentiment_loss, summary_loss_weight, sentiment_loss_weight
    total_training_loss = []
    total_summary_loss = []
    total_sentiment_loss = []
    same_sentiment_count = 0
    total_count = 0
    summary_model.train()
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    for news, summaries in pbar:
        if news is None or summaries is None:
            print("Error: News or Summaries is None")
            continue
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))
        if 'input_ids' not in inputs or 'attention_mask' not in inputs:
            print(f"Error: Invalid inputs from data_collator: {inputs}")
            continue
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)

        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
        if 'input_ids' not in labels:
            print(f"Error: Invalid labels from data_collator: {labels}")
            continue
        y_input_ids = labels['input_ids'].to(DEVICE)

        summary_loss = summary_model(input_ids=X_input_ids, attention_mask=X_attention_mask, labels=y_input_ids).loss

        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)
        if not gen_summaries:
            print("Error: Generated summaries are empty")
            continue

        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        news_sentiment_probs = labels_to_probs(news_sentiment_labels, len(SENTIMENT_LABELS)).to(DEVICE)
        news_sentiment_logits = probs_to_logits(news_sentiment_probs)

        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        gen_summaries_sentiment_logits = probs_to_logits(gen_summaries_sentiment_probs).to(DEVICE)

        if news_sentiment_logits.size(0) == 0 or gen_summaries_sentiment_logits.size(0) == 0:
            print("Error: Sentiment labels or logits are empty")
            continue

        sentiment_loss = loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

        # 計算相同情感的比例
        same_sentiment_count += (torch.argmax(news_sentiment_logits, dim=1) == torch.argmax(gen_summaries_sentiment_logits, dim=1)).sum().item()
        total_count += news_sentiment_labels.size(0)

        # 標準化損失
        standardized_summary_loss = standardize_loss(summary_loss.item(), avg_summary_loss)
        standardized_sentiment_loss = standardize_loss(sentiment_loss.item(), avg_sentiment_loss)

        # 確保標準化后的損失是浮點數
        standardized_summary_loss = torch.tensor(float(standardized_summary_loss), dtype=torch.float32, requires_grad=True).to(DEVICE)
        standardized_sentiment_loss = torch.tensor(float(standardized_sentiment_loss), dtype=torch.float32, requires_grad=True).to(DEVICE)

        total_loss = summary_loss_weight * standardized_summary_loss + sentiment_loss_weight * standardized_sentiment_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_training_loss.append(total_loss.item())
        total_summary_loss.append(summary_loss.item())
        total_sentiment_loss.append(sentiment_loss.item())

        # 更新進度條的後綴信息，顯示當前批次損失和平均損失
        pbar.set_postfix_str('Batch Loss: {:.6f}, Avg Loss: {:.6f}'.format(total_loss.item(), sum(total_training_loss) / len(total_training_loss)))

    avg_summary_loss = 0.9 * avg_summary_loss + 0.1 * (sum(total_summary_loss) / len(total_summary_loss))
    avg_sentiment_loss = 0.9 * avg_sentiment_loss + 0.1 * (sum(total_sentiment_loss) / len(total_sentiment_loss))
    same_sentiment_rate = same_sentiment_count / total_count
    print(f"Same Sentiment Rate: {same_sentiment_rate:.3f}")
    return avg_summary_loss, avg_sentiment_loss, same_sentiment_rate

**5. 直接用logits算loss**

In [22]:
def train_step(epoch, avg_summary_loss, avg_sentiment_loss):
    global prev_summary_loss, prev_sentiment_loss, summary_loss_weight, sentiment_loss_weight
    total_training_loss = []
    total_summary_loss = []
    total_sentiment_loss = []
    same_sentiment_count = 0
    total_count = 0
    summary_model.train()
    pbar = tqdm(train_dataloader)
    pbar.set_description_str('[Epoch {}] Training'.format(epoch))

    for news, summaries in pbar:
        if news is None or summaries is None:
            print("Error: News or Summaries is None")
            continue
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))
        if 'input_ids' not in inputs or 'attention_mask' not in inputs:
            print(f"Error: Invalid inputs from data_collator: {inputs}")
            continue
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)

        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))
        if 'input_ids' not in labels:
            print(f"Error: Invalid labels from data_collator: {labels}")
            continue
        y_input_ids = labels['input_ids'].to(DEVICE)

        summary_loss = summary_model(input_ids=X_input_ids, attention_mask=X_attention_mask, labels=y_input_ids).loss

        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)
        if not gen_summaries:
            print("Error: Generated summaries are empty")
            continue

        # 獲取新聞的情緒標籤索引
        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        # 獲取生成的摘要的情緒logits
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs.clamp(min=1e-9))

        # 確保 logits 沒有問題
        if gen_summaries_sentiment_logits.size(0) == 0 or news_sentiment_labels.size(0) == 0:
            print("Error: Sentiment labels or logits are empty")
            continue

        # 使用 CrossEntropyLoss 計算情緒損失
        sentiment_loss = loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

        # 計算相同情感的比例
        same_sentiment_count += (torch.argmax(gen_summaries_sentiment_logits, dim=1) == news_sentiment_labels).sum().item()
        total_count += news_sentiment_labels.size(0)

        # 標準化損失
        standardized_summary_loss = standardize_loss(summary_loss.item(), avg_summary_loss)
        standardized_sentiment_loss = standardize_loss(sentiment_loss.item(), avg_sentiment_loss)

        # 確保標準化后的損失是浮點數
        standardized_summary_loss = torch.tensor(float(standardized_summary_loss), dtype=torch.float32, requires_grad=True).to(DEVICE)
        standardized_sentiment_loss = torch.tensor(float(standardized_sentiment_loss), dtype=torch.float32, requires_grad=True).to(DEVICE)

        total_loss = summary_loss_weight * standardized_summary_loss + sentiment_loss_weight * standardized_sentiment_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_training_loss.append(total_loss.item())
        total_summary_loss.append(summary_loss.item())
        total_sentiment_loss.append(sentiment_loss.item())

        # 更新進度條的後綴信息，顯示當前批次損失和平均損失
        pbar.set_postfix_str('Batch Loss: {:.6f}, Avg Loss: {:.6f}'.format(total_loss.item(), sum(total_training_loss) / len(total_training_loss)))

    avg_summary_loss = 0.9 * avg_summary_loss + 0.1 * (sum(total_summary_loss) / len(total_summary_loss))
    avg_sentiment_loss = 0.9 * avg_sentiment_loss + 0.1 * (sum(total_sentiment_loss) / len(total_sentiment_loss))
    same_sentiment_rate = same_sentiment_count / total_count
    print(f"Same Sentiment Rate: {same_sentiment_rate:.3f}")
    return avg_summary_loss, avg_sentiment_loss, same_sentiment_rate

**執行單個驗證步驟（epoch）。**

*   設置模型為評估模式。
*   遍歷驗證數據集，對每個批次進行處理：
    *   生成模型輸入。
    *   計算損失，包括生成摘要的損失和情感匹配損失。
    *   記錄和打印損失。

主要作用是評估模型在驗證數據集上的性能。

In [10]:
def eval_step(epoch):
    total_vaild_loss = []  # 初始化存儲總驗證損失的列表

    summary_model.eval()  # 設置模型為評估模式

    pbar = tqdm(eval_dataloader)  # 初始化進度條
    # 設置進度條的描述，顯示當前的驗證週期
    pbar.set_description_str('[Epoch {}] Validation'.format(epoch))

    # 驗證數據加載器中的每個批次
    for news, summaries in pbar:
        # 新聞文本轉換為模型輸入格式
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))

        # 模型的輸入數據 ID 和注意力掩碼移動到設備（例如 GPU）。
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)  # 移動注意力掩碼到設備

        # 摘要轉換為模型的目標輸入格式，並移動到設備上
        # labels 的輸出張量
          # input_ids：轉換後的摘要文本的標記 ID。
          # attention_mask：填充的位置，指示哪些位置是填充（0），哪些是有效標記（1）。
        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))  # 獲取目標數據
        y_input_ids = labels['input_ids'].to(DEVICE)  # 移動目標輸入 ID 到設備

        # 計算"新聞文本與對應摘要"的損失函數。
        loss = summary_model(input_ids=X_input_ids,
                             attention_mask=X_attention_mask,
                             labels=y_input_ids).loss

        # 生成摘要
        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)

        # 獲取新聞的情感標籤
        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        # 獲取摘要的情感概率

        '''get_sentiment_probs輸出是機率，不符CrossEntropyLoss 的預期輸入（即 logits）'''
        # gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        '''為了要符合 CrossEntropyLoss 的預期輸入（即 logits），get_sentiment_logits返回logits'''
        # gen_summaries_sentiment_logits = get_sentiment_logits(sentiment_model, gen_summaries).to(DEVICE)
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        # .clamp(min=1e-9) 是一個張量操作，將 gen_summaries_sentiment_probs 的值限制在至少 1×10−9的範圍內。防止概率值過小（接近於零），以避免在後續計算中出現數值不穩定或錯誤。
        gen_summaries_sentiment_probs = gen_summaries_sentiment_probs.clamp(min=1e-9)
        gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs)

        # 加上情感匹配損失
        # 找到新聞情感的對應摘要機率，帶入損失函數做計算
        # 原本loss生成摘要與新聞的損失
        # loss_fcn基於"生成摘要的情感分布"與"原始新聞文本的情感標籤"之間的差異。
        '''輸入方式是機率，不符CrossEntropyLoss 的預期輸入（即 logits）
        loss = loss + loss_fcn(gen_summaries_sentiment_probs, news_sentiment_labels)'''
        loss = loss + loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

        total_vaild_loss.append(loss.item())  # 記錄損失

        # 更新進度條後綴，顯示當前批次的損失和平均損失
        pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(loss.item(), sum(total_vaild_loss) / len(total_vaild_loss)))




**5.**

In [23]:
def eval_step(epoch):
    total_vaild_loss = []  # 初始化存儲總驗證損失的列表

    summary_model.eval()  # 設置模型為評估模式

    pbar = tqdm(eval_dataloader)  # 初始化進度條
    # 設置進度條的描述，顯示當前的驗證週期
    pbar.set_description_str('[Epoch {}] Validation'.format(epoch))

    # 驗證數據加載器中的每個批次
    for news, summaries in pbar:
        # 新聞文本轉換為模型輸入格式
        inputs = data_collator(get_model_inputs(summary_tokenizer, news))

        # 模型的輸入數據 ID 和注意力掩碼移動到設備（例如 GPU）。
        X_input_ids = inputs['input_ids'].to(DEVICE)
        X_attention_mask = inputs['attention_mask'].to(DEVICE)  # 移動注意力掩碼到設備

        # 摘要轉換為模型的目標輸入格式，並移動到設備上
        # labels 的輸出張量
          # input_ids：轉換後的摘要文本的標記 ID。
          # attention_mask：填充的位置，指示哪些位置是填充（0），哪些是有效標記（1）。
        labels = data_collator(get_model_inputs(summary_tokenizer, summaries, is_summaries=True))  # 獲取目標數據
        y_input_ids = labels['input_ids'].to(DEVICE)  # 移動目標輸入 ID 到設備

        # 計算"新聞文本與對應摘要"的損失函數。
        loss = summary_model(input_ids=X_input_ids,
                             attention_mask=X_attention_mask,
                             labels=y_input_ids).loss

        # 生成摘要
        gen_summaries = summary_tokenizer.batch_decode(summary_model.generate(X_input_ids), skip_special_tokens=True)

        # 獲取新聞的情感標籤索引
        news_sentiment_labels = get_sentiment_labels(sentiment_model, news).to(DEVICE)
        # 獲取摘要的情感logits
        gen_summaries_sentiment_probs = get_sentiment_probs(sentiment_model, gen_summaries).to(DEVICE)
        gen_summaries_sentiment_logits = torch.log(gen_summaries_sentiment_probs.clamp(min=1e-9))

        # 確保 logits 沒有問題
        if gen_summaries_sentiment_logits.size(0) == 0 or news_sentiment_labels.size(0) == 0:
            print("Error: Sentiment labels or logits are empty")
            continue

        # 使用 CrossEntropyLoss 計算情緒損失
        sentiment_loss = loss_fcn(gen_summaries_sentiment_logits, news_sentiment_labels)

        # 加上情感匹配損失
        loss = loss + sentiment_loss

        total_vaild_loss.append(loss.item())  # 記錄損失

        # 更新進度條後綴，顯示當前批次的損失和平均損失
        pbar.set_postfix_str('Batch Loss: {:.6f}, Average Loss: {:.6f}'.format(loss.item(), sum(total_vaild_loss) / len(total_vaild_loss)))

**從模型檔案名稱中提取 epoch 編號。**

返回提取出的 epoch 編號（整數）。

In [24]:
# 定義一個函數用來從檔案名中解析出模型訓練的 epoch 數字
def get_model_epoch_num(file_name):
    # 從檔案名分割出 epoch 數字，並轉換成整數型態返回
    '''
    model_epoch_10.pt
    file_name.split('_') ["model", "epoch", "10.pt"]
    file_name.split('_')[-1]  "10.pt"
    file_name.split('_')[-1][:-3]  "10"
    '''
    return int(file_name.split('_')[-1][:-3])

# 用來存儲最新的模型檢查點的字典
ckpt = None

# 檢查模型儲存路徑是否存在，不存在則創建
if not os.path.exists(MODEL_SAVE_PTH):
    os.mkdir(MODEL_SAVE_PTH)

# 檢查指定路徑下是否有檔案存在
if len(os.listdir(MODEL_SAVE_PTH)) != 0:
    latest = os.listdir(MODEL_SAVE_PTH)[0]  # 先預設第一個檔案為最新

    # 遍歷該路徑下的所有檔案
    for file in os.listdir(MODEL_SAVE_PTH):
        # 檢查檔案是否為 PyTorch 的模型檔案（以 'pt' 結尾），且檢查是否比已知的最新檔案新
        # file[-2:]:倒數第二個字符開始一直到字符串的結尾
        if file[-2:] == 'pt' and get_model_epoch_num(file) > get_model_epoch_num(latest):
            latest = file  # 更新最新檔案

    # 加載最新的模型檢查點
    # 模型保存路徑 MODEL_SAVE_PTH 和最新的檔案名 latest 組合成一個完整的文件路徑。
    '''
    latest 是最新的檢查點"檔案名稱"
    MODEL_SAVE_PTH 是保存"檔案的目錄"
    torch.load 函數將檢查點檔案加載到 ckpt 變數中，這是一個包含模型狀態和訓練進度的字典。
    '''
    ckpt = torch.load(os.path.join(MODEL_SAVE_PTH, latest))

start_epoch = 0  # 初始化訓練開始的 epoch 數

# 如果 ckpt 不是 None，即成功加載了模型檢查點
if ckpt != None:
    # 恢復訓練進度，可以從上次訓練結束的地方繼續訓練
    start_epoch = ckpt['epoch']  # 從檢查點讀取目前的 epoch 數
    # load_state_dict 將檢查點中的模型參數(權重、biases)，加載到 summary_model 模型中
    summary_model.load_state_dict(ckpt['model_state_dict'])


KeyboardInterrupt: 

**全部印出來**

In [25]:
# 這裡可以設定epoch要從哪裡開始
# 假設我之前訓練到第5個epoch，那我還要繼續10個range，我從這裏的start_epoch改掉就好
# start_epoch=5, range=10

# 定義一個函數來獲取模型檔案的訓練 epoch 編號
def get_model_epoch_num(file_name):
    # 假設檔案名稱格式為 "model_epoch_XX.pt"，提取最後的數字部分作為 epoch 編號
    return int(file_name.split('_')[-1][:-3])

# 初始化模型檢查點為 None
# 變數 ckpt 是用來存儲最新加載的模型檢查點的。模型檢查點通常包含了訓練過程中的關鍵狀態，
# 例如當前的 epoch 編號、模型的參數（狀態字典）和優化器的狀態等。
ckpt = None

# 如果模型保存路徑不存在，則創建該目錄
if not os.path.exists(MODEL_SAVE_PTH):
    os.mkdir(MODEL_SAVE_PTH)

# 如果模型保存路徑下有檔案
if len(os.listdir(MODEL_SAVE_PTH)) != 0:
    # 初始化 latest變數為_目錄中的第一個檔案
    latest = os.listdir(MODEL_SAVE_PTH)[0]
    print(latest) #state_dict_1.pt
    # 遍歷模型保存路徑中的所有檔案
    for file in os.listdir(MODEL_SAVE_PTH):
        # 檢查檔案是否為模型檔案（假設模型檔案的副檔名為 .pt）
        if file[-2:] == 'pt':
            # 更新 assign 變數為找到的 state_dict_5.pt 檔案
            if file == 'state_dict_1.pt':
                assign = file
            # 如果該檔案的 epoch 編號大於目前的最新檔案，則更新 latest
            if get_model_epoch_num(file) > get_model_epoch_num(latest):
                latest = file

    print(latest) #state_dict_5.pt
    print(assign)
    # 加載最新的模型檢查點
    # torch.save 和 torch.load 來保存和加載模型的參數狀態字典
    #ckpt = torch.load(os.path.join(MODEL_SAVE_PTH, latest))
    ckpt = torch.load(os.path.join(MODEL_SAVE_PTH, assign)) #我要從第5個epoch開始跑看看
    # print(ckpt)

# 初始化開始的 epoch 為 0
start_epoch = 0

# 如果找到了模型檢查點
if ckpt != None:
    # 更新start_epoch 為檢查點中保存的epoch
    start_epoch = ckpt['epoch']
    # 加載檢查點中保存的模型狀態字典
    summary_model.load_state_dict(ckpt['model_state_dict'])

print(start_epoch)

state_dict_1.pt
state_dict_2.pt
state_dict_1.pt
1


In [ ]:
# 在訓練開始之前進行一次測試步驟，確定模型的初始性能
# 了解訓練資料集摘要與新聞之間的情緒關係
test_step()

# 循環遍歷每一個訓練周期（epoch）
# 從上次保存的 epoch 開始，進行訓練、驗證和測試。
# start_epoch+1 表示從下一個 epoch 開始
# start_epoch+num_train_epochs+1 確保總共訓練的 epoch 總數正確。
for epoch in range(start_epoch+1, start_epoch+num_train_epochs+1):
    train_step(epoch)  # 執行訓練步驟，傳入當前的 epoch 數，用於訓練模型
    eval_step(epoch)   # 執行評估步驟，通常用於在驗證集上評估模型的性能
    test_step()        # 執行測試步驟，用於在測試集上評估模型的性能

    # 保存當前 epoch 的模型檢查點
    torch.save({
        'epoch': epoch,  # 存儲當前的 epoch 數
        'model_state_dict': summary_model.state_dict()  # 存儲模型的權重和偏置
    }, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))  # 檢查點保存的文件名及路徑


  6%|▌         | 4/67 [00:07<02:04,  1.97s/it]


KeyboardInterrupt: 

**有調整loss權重的**

In [21]:
test_step()

for epoch in range(start_epoch+1, start_epoch+num_train_epochs+1):
    summary_loss_weight, sentiment_loss_weight = adjust_weights(prev_summary_loss, prev_sentiment_loss)
    train_step(epoch)
    eval_step(epoch)
    test_step()
    torch.save({
        'epoch': epoch,
        'model_state_dict': summary_model.state_dict()
    }, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))


 16%|█▋        | 11/67 [00:18<01:36,  1.72s/it]


KeyboardInterrupt: 

**3. 4.標準化損失和動態調整權重**

In [ ]:
def check_sentiment_distribution(corpus):
    sentiment_labels = []
    for text in tqdm(corpus, desc="Processing corpus for sentiment distribution"):
        labels = get_sentiment_labels(sentiment_model, [text])
        sentiment_labels.extend(labels.tolist())

    sentiment_counts = Counter(sentiment_labels)
    print("Sentiment distribution:")
    for label, count in sentiment_counts.items():
        print(f"{label}: {count}")
    return sentiment_counts

# 在訓練開始之前進行一次測試步驟，確定模型的初始性能
test_step()

# 檢查訓練集中的情感標籤分布
all_news = []
for news, _ in train_dataloader:
    all_news.extend(news)

check_sentiment_distribution(all_news)

# 循環遍歷每一個訓練周期（epoch）
for epoch in range(start_epoch+1, start_epoch+num_train_epochs+1):
    summary_loss_weight, sentiment_loss_weight = adjust_weights(avg_summary_loss, avg_sentiment_loss)
    avg_summary_loss, avg_sentiment_loss = train_step(epoch, avg_summary_loss, avg_sentiment_loss)
    eval_step(epoch)
    test_step()
    torch.save({
        'epoch': epoch,
        'model_state_dict': summary_model.state_dict()
    }, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))

Processing corpus for sentiment distribution: 100%|██████████| 1192/1192 [08:31<00:00,  2.33it/s]


Sentiment distribution:
5: 403
2: 587
3: 65
0: 92
4: 23
1: 20
6: 2


[Epoch 1] Training:   0%|          | 0/149 [00:05<?, ?it/s]


TypeError: object of type 'NoneType' has no len()

In [ ]:
for epoch in range(start_epoch+1, start_epoch+num_train_epochs+1):
    summary_loss_weight, sentiment_loss_weight = adjust_weights(avg_summary_loss, avg_sentiment_loss)
    avg_summary_loss, avg_sentiment_loss, same_sentiment_rate = train_step(epoch, avg_summary_loss, avg_sentiment_loss)
    eval_step(epoch)
    test_step()
    torch.save({
        'epoch': epoch,
        'model_state_dict': summary_model.state_dict()
    }, os.path.join(MODEL_SAVE_PTH, f'state_dict_{epoch}.pt'))

[Epoch 1] Training: 100%|██████████| 250/250 [29:16<00:00,  7.02s/it, Batch Loss: 1.573619, Avg Loss: 1.262937]


Same Sentiment Rate: 0.453


[Epoch 2] Training:  62%|██████▏   | 154/250 [17:49<10:49,  6.76s/it, Batch Loss: 9.476467, Avg Loss: 8.897231]

**6. 全部印出來**

In [28]:
# 最後一區有用到 __測試集

# 定義一個函數來在測試集上評估模型
def test_step():
    # 將模型設置為評估模式
    summary_model.eval()

    # 初始化一個列表來存儲測試結果
    result = []

    # 初始化一個列表來存儲測試結果
    result_new = []

    # 初始化進度條
    pbar = tqdm(test_dataloader)

    # 遍歷測試數據集中的每個批次
    for index, (news, _) in enumerate(pbar):
        # 使用 data_collator 函數和分詞器處理新聞文本，生成模型輸入
        summary_model_inputs = data_collator(get_model_inputs(summary_tokenizer, news))['input_ids'].to(DEVICE)
        # 使用模型生成摘要
        summary_model_outputs = summary_model.generate(summary_model_inputs)
        # 將生成的摘要轉換為文本
        summarized_text = summary_tokenizer.batch_decode(summary_model_outputs, skip_special_tokens=True)

        # 打印每個生成的摘要
        print(f'news Text {index + 1}: {news}')

        # 打印每個生成的摘要
        print(f'Summarized Text {index + 1}: {summarized_text}')

                # 將結果添加到結果列表中
        result_new.append({
            'News': news[0],
            'Summarized Text': summarized_text,

        })

        # 使用情緒分析模型獲取新聞文本的情緒標籤
        news_sentiment_label = get_sentiment_labels(sentiment_model, news)[0]
        # 使用情緒分析模型獲取生成摘要的情緒標籤
        summarized_sentiment_label = get_sentiment_labels(sentiment_model, summarized_text)[0]

        # 比較新聞文本和生成摘要的情緒標籤是否相同，並將結果添加到結果列表中
        # result是一個列表來存儲測試結果 如果摘要和原文標籤相同就append1，最後計算1總共的數量
        result.append(1 if news_sentiment_label == summarized_sentiment_label else 0)

        # 如果當前批次是最後一個，更新進度條的後綴信息，顯示相同情緒的比例
        # 所以是1越多越好
        # Same Sentiment Rate: 摘要和原文標籤相同/總共原文數量
        if index == len(test_dataloader) - 1:
            pbar.set_postfix_str('Same Sentiment Rate: {:.3f}'.format(result.count(1) / len(result)))
        # 創建 pandas DataFrame 並顯示表格
    results_df = pd.DataFrame(result_new)
    print(results_df)

    # 將結果匯出成 CSV 文件
    results_df.to_csv('results.csv', index=False)

In [ ]:
test_step()

  0%|          | 0/112 [00:00<?, ?it/s]

news Text 1: ['UK house prices dip in November  UK house prices dipped slightly in November, the Office of the Deputy Prime Minister (ODPM) has said.  The average house price fell marginally to Â£180,226, from Â£180,444 in October. Recent evidence has suggested that the UK housing market is slowing after interest rate increases, and economists forecast a drop in prices during 2005. But while the monthly figures may hint at a cooling of the market, annual house price inflation is still strong, up 13.8% in the year to November. Economists, however, forecast that ODPM figures are likely to show a weakening in annual house price growth in coming months. "Overall, the housing market activity is slowing down and that is backed up by the mortgage lending and the mortgage approvals data," said Mark Miller, at HBOS Treasury Services. "The ODPM data is a fairly lagging indicator."  The figures come after the Bank of England said the number of mortgages approved in the UK has fallen to the lowest

  1%|          | 1/112 [00:01<02:42,  1.46s/it]

news Text 2: ['LSE \'sets date for takeover deal\'  The London Stock Exchange (LSE) is planning to announce a preferred takeover by the end of the month, newspaper reports claim.  The Sunday Telegraph said the LSE\'s plan was further evidence it wants to retain tight control over its destiny. Both Deutsche Boerse and rival Euronext held talks with the London market last week over a possible offer. A Â£1.3bn offer from Deutsche Boerse has already been rejected, while Euronext has said it will make an all cash bid. Speculation suggests that Paris-based Euronext has the facilities in place to make a bid of Â£1.4bn, while its German rival may up its bid to the Â£1.5bn mark. Neither has yet tabled a formal bid, but the LSE is expected to hold further talks with the two parties later this week. However, the Sunday Telegraph report added that there are signs that Deutsche Boerse chief executive Werner Seifert is becoming increasingly impatient with the LSE\'s managed bid process.  Despite ins

  2%|▏         | 2/112 [00:02<02:40,  1.46s/it]

...

news Text 77: ["Man Utd through after Exeter test  Manchester United avoided an FA Cup upset by edging past Exeter City in their third round replay.  Cristiano Ronaldo scored the opener, slipping the ball between Paul Jones' legs after just nine minutes. United wasted a host of chances to make it safe as Jones made some great saves, but Wayne Rooney put the tie beyond doubt late on with a cool finish. Exeter had chances of their own, Sean Devine twice volleying wide and Andrew Taylor forcing Tim Howard to save. United boss Sir Alex Ferguson was taking few chances after their 0-0 draw in the first game and he handed starts to Paul Scholes and Ryan as well as Ronaldo and Rooney. Exeter began brightly with Devine and Steve Flack seeing plenty of the ball, but it did not take United long to assert their authority and the hosts soon found themselves a goal down. Scholes played a lovely pass in to Ronaldo on the left-hand side of the six-yard box and the Portuguese winger slid the ball betwe

 69%|██████▉   | 77/112 [01:48<00:49,  1.42s/it]

news Text 78: ['Casino Royale is next Bond movie  Casino Royale, author Ian Fleming\'s first James Bond book, is to be the next Bond film, with Goldeneye director Martin Campbell behind the camera.  It will be the 21st James Bond film to hit the big screen, and speculation has been rife over who will play the lead. Casino Royale was turned into a spoof spy movie by John Huston in 1967, with David Niven in the lead role. Pierce Brosnan led the past four Bond films but said producers axed him after offering him the chance to return. Among the favourites to take over the coveted role are Scottish actor Dougray Scott, Oscar nominee Clive Owen and Australian star Hugh Jackman. Producers say no decision has yet been made on who will become the seventh actor, including Niven, to play Bond on film. Kill Bill director Quentin Tarantino had talked of wanting to take on the Casino Royale project, and said he had spoken to Brosnan about it.  Shooting on Casino Royale is expected to begin once Camp

 70%|██████▉   | 78/112 [01:50<00:48,  1.43s/it]

news Text 79: ['Feta cheese battle reaches court  A row over whether only Greece should be allowed to label its cheese feta has reached the European Court of Justice.  The Danish and German governments are challenging a European Commission ruling which said Greece should have sole rights to use the name. The Commission\'s decision gave the same legal protection to feta as to Italian Parma ham and French Champagne. But critics of the judgement say feta is a generic term, with the cheese produced widely outside Greece.  The Commission\'s controversial 2002 ruling gave "protected designation of origin" status to feta cheese made in Greece, effectively restricting the use of the feta name to producers there.  From 2007 onwards, Greek firms will have the exclusive use of the feta label and producers elsewhere in Europe must find another name to describe their products. The German and Danish governments argue that feta does not relate to a specific geographical area and that their firms have